In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import requests
import pandas as pd

API_KEY = "9ZzmOMQcT5MRCTnuC0Dw"
url = f"https://theyvoteforyou.org.au/api/v1/policies.json?key={API_KEY}"

# Call API
response = requests.get(url)
data = response.json()

# Convert to Dataframe
df = pd.DataFrame(data)

# key words
include_keywords = ["forest", "logging", "conservation", "threatened species", "wildlife", "habitat"]
exclude_keywords = ["marine", "sealife"]

# Filter policies with key words
filtered = df[
    df["name"].str.lower().str.contains('|'.join(include_keywords)) |
    df["description"].str.lower().str.contains('|'.join(include_keywords))
]
filtered = filtered[
    ~filtered["name"].str.lower().str.contains('|'.join(exclude_keywords)) &
    ~filtered["description"].str.lower().str.contains('|'.join(exclude_keywords))
]

print(filtered[["id", "name", "description"]])
csv_output_path = "/content/drive/MyDrive/FIT5120/Main_project/Datascience/environmental_policies.csv"
filtered[["id", "name", "description"]].to_csv(csv_output_path, index=False)

      id                                               name  \
51    52  reducing planned economic immigration for econ...   
82    83  letting environmental groups challenge the leg...   
92    93                             ending illegal logging   
93    94  protecting threatened forest and bushland habi...   
171  172  federal government action on animal & plant ex...   
216  217                        Australia's timber industry   
330  331              placing the CFMEU into administration   

                                           description  
51   Federal and State governments have a duty to e...  
82   environmental and conservation groups should b...  
92   the federal government should introduce legisl...  
93   the federal government should protect threaten...  
171  the federal government needs to support resear...  
216  the federal government should protect Australi...  
330  the federal government should put the Construc...  


In [ ]:
policy_ids = filtered["id"].tolist()
print(policy_ids)

[52, 83, 93, 94, 172, 217, 331]


In [ ]:
for pid in policy_ids:
    url = f"https://theyvoteforyou.org.au/api/v1/policies/{pid}.json?key=9ZzmOMQcT5MRCTnuC0Dw"
    response = requests.get(url)
    data = response.json()
    print(f"--- Policy ID: {pid} ---")
    print(data)

--- Policy ID: 52 ---
{'id': 52, 'name': 'reducing planned economic immigration for economic and environmental reasons', 'description': 'Federal and State governments have a duty to ensure training, higher education, employment and housing are provided to citizens as a priority over invited economic immigrants.  Furthermore that Australia should keep its population small in light of\r\n(1) - diminishing natural space and habitat for Australian plants and animals\r\n(2) - demand-driven housing unaffordability and developer-driven population growth policies impacting democracy and quality of life\r\n(3) - demand-driven resource unaffordability and scarcity, including water, fuel and power\r\n(4) - infrastructure overuse\r\n(5) - diseconomies of scale\r\n(6) - associated cost of living rises', 'provisional': True, 'last_edited_at': '2014-10-28T17:07:27+11:00', 'policy_divisions': [], 'people_comparisons': []}
--- Policy ID: 83 ---
{'id': 83, 'name': 'letting environmental groups challenge

In [ ]:
import requests
import pandas as pd

# --- Configuration ---
API_KEY = "9ZzmOMQcT5MRCTnuC0Dw"
all_supporters = []

for policy_id in policy_ids:
    url = f"https://theyvoteforyou.org.au/api/v1/policies/{policy_id}.json?key={API_KEY}"
    response = requests.get(url)
    data = response.json()

    policy_name = data.get("name", "")
    policy_desc = data.get("description", "")
    people_comparisons = data.get("people_comparisons", [])

    df = pd.DataFrame(people_comparisons)

    # Skip if empty
    if df.empty:
        continue

    # Extract nested fields safely
    df['first_name'] = df['person'].apply(lambda x: x.get('latest_member', {}).get('name', {}).get('first', '') if isinstance(x, dict) else '')
    df['last_name'] = df['person'].apply(lambda x: x.get('latest_member', {}).get('name', {}).get('last', '') if isinstance(x, dict) else '')
    df['full_name'] = df['first_name'] + ' ' + df['last_name']
    df['electorate'] = df['person'].apply(lambda x: x.get('latest_member', {}).get('electorate', '') if isinstance(x, dict) else '')
    df['party'] = df['person'].apply(lambda x: x.get('latest_member', {}).get('party', '') if isinstance(x, dict) else '')
    df['house'] = df['person'].apply(lambda x: x.get('latest_member', {}).get('house', '') if isinstance(x, dict) else '')

    # Clean dataframe
    df_clean = df[['full_name', 'electorate', 'party', 'house', 'agreement', 'voted', 'category']]
    df_clean['agreement'] = pd.to_numeric(df_clean['agreement'], errors='coerce')

    # Filter for Victoria + strong supporters
    df_vic = df_clean[df_clean['electorate'].str.contains('victoria', case=False, na=False)]
    df_vic_supporters = df_vic[df_vic['agreement'] >= 60].copy()

    # Add policy info to the filtered records
    df_vic_supporters['policy_id'] = policy_id
    df_vic_supporters['policy_name'] = policy_name
    df_vic_supporters['policy_description'] = policy_desc

    all_supporters.append(df_vic_supporters)

# Combine all results
final_df = pd.concat(all_supporters, ignore_index=True)

# Show result
print(final_df)

# Save to CSV (optional)
final_df.to_csv("/content/drive/MyDrive/FIT5120/Main_project/Datascience/vic_policy_supporters.csv", index=False)

<ipython-input-7-a29b0632f9a5>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['agreement'] = pd.to_numeric(df_clean['agreement'], errors='coerce')
<ipython-input-7-a29b0632f9a5>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['agreement'] = pd.to_numeric(df_clean['agreement'], errors='coerce')
<ipython-input-7-a29b0632f9a5>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

            full_name electorate                   party   house  agreement  \
0            Kim Carr   Victoria  Australian Labor Party  senate      100.0   
1     Jacinta Collins   Victoria  Australian Labor Party  senate      100.0   
2   Richard Di Natale   Victoria       Australian Greens  senate      100.0   
3          Janet Rice   Victoria       Australian Greens  senate      100.0   
4      Stephen Conroy   Victoria  Australian Labor Party  senate       92.0   
..                ...        ...                     ...     ...        ...   
56         Jess Walsh   Victoria  Australian Labor Party  senate      100.0   
57          David Van   Victoria             Independent  senate      100.0   
58        Ralph Babet   Victoria  United Australia Party  senate      100.0   
59       Jana Stewart   Victoria  Australian Labor Party  senate      100.0   
60      Lisa Darmanin   Victoria  Australian Labor Party  senate      100.0   

    voted    category  policy_id  \
0    True      

<ipython-input-7-a29b0632f9a5>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['agreement'] = pd.to_numeric(df_clean['agreement'], errors='coerce')


In [ ]:
df_supporters_clean = final_df.drop(columns=['policy_name', 'policy_description'])
df_supporters_clean.to_csv("/content/drive/MyDrive/FIT5120/Main_project/Datascience/policy_supporters_clean.csv", index=False)

In [ ]:
!pip install psycopg2-binary pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 38.5 MB/s eta 0:00:00


In [ ]:
import psycopg2
import pandas as pd
import json
from collections import defaultdict

# --- Supabase Postgres connection ---
conn = psycopg2.connect(
    host="aws-0-ap-southeast-2.pooler.supabase.com",
    port=6543,
    dbname="postgres",
    user="postgres.tchwwnlazebhayosesvf",
    password="ta28_monash"
)

# --- Query only required fields ---
query = """
SELECT
    p.id AS policy_id,
    p.name AS policy_name,
    p.description AS policy_description,
    s.full_name,
    s.party,
    s.electorate
FROM policy_supporters s
JOIN policies p ON s.policy_id = p.id
ORDER BY p.id, s.agreement DESC;
"""

df = pd.read_sql_query(query, conn)
conn.close()

# --- Group supporters by policy ---
grouped = defaultdict(lambda: {"policy_name": "", "policy_description": "", "supporters": []})

for _, row in df.iterrows():
    pid = int(row["policy_id"])
    grouped[pid]["policy_name"] = str(row["policy_name"])
    grouped[pid]["policy_description"] = str(row["policy_description"])
    grouped[pid]["supporters"].append({
        "full_name": str(row["full_name"]),
        "party": str(row["party"]),
        "electorate": str(row["electorate"])
    })

# --- Prepare final JSON structure ---
output = []
for pid, content in grouped.items():
    output.append({
        "policy_id": pid,
        "policy_name": content["policy_name"],
        "policy_description": content["policy_description"],
        "supporters": content["supporters"]
    })

# --- Save to JSON file ---
with open("/content/drive/MyDrive/FIT5120/Main_project/Datascience/policy_cards_minimal.json", "w", encoding="utf-8") as f:
    json.dump(output, f, indent=2, ensure_ascii=False)

print("✅ Saved to policy_cards_minimal.json")

<ipython-input-14-a99d37551bf4>:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


✅ Saved to policy_cards_minimal.json


In [ ]:
import requests

# Define the API endpoint with the desired activity category
api_url = "https://www.volunteerconnector.org/api/search/?ac=5"

# Send a GET request to the API
response = requests.get(api_url)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    # Iterate through the results and print relevant information
    for opportunity in data.get('results', []):
        print(f"Title: {opportunity.get('title')}")
        print(f"Description: {opportunity.get('description')}")
        print(f"URL: {opportunity.get('url')}\n")
else:
    print(f"Error: Received status code {response.status_code}")


Title: Diversity Awareness and Skill Building Youth Program - Changemakers
Description: Are you between 16 - 30 years of age, and passionate about Diversity and creating Inclusive Communities? 

The Diversity Awareness Program will offer you valuable opportunities to develop your skills and knowledge about Diversity, Equity and Inclusion, connections to volunteer for an organization in your community that promotes these values, and support to develop a capstone project to promote diversity and inclusion in your own community. 

The Diversity Awareness and Skills Building Youth Program is designed to set the stage for for all participants, including an emphasis on those who identify as a members of an equity-deserving group, to become advocates for yourselves and allies for others. 

Enhance your knowledge and understanding of barriers to inclusion at school or in the workplace, while building your leadership skills. Become a leader in promoting diversity and inclusion in your communiti

In [ ]:
import requests

# --- Cấu hình ---
api_url = "https://www.volunteerconnector.org/api/search/?ac=5&ac=131&ac=59&cc=64"  # Thêm nhiều mã category nếu cần

# Từ khóa liên quan đến môi trường
env_keywords = ["Environment", "conservation", "wildlife", "forest", "habitat", "sustainability"]

# Gửi yêu cầu
response = requests.get(api_url)

if response.status_code == 200:
    data = response.json()
    results = data.get("results", [])

    for opportunity in results:
        activities = opportunity.get("activities", [])

        # Kiểm tra nếu bất kỳ hoạt động nào chứa từ khóa môi trường
        is_environmental = any(
            any(keyword in (act.get("name", "") + act.get("category", "")).lower() for keyword in env_keywords)
            for act in activities
        )

        if is_environmental:
            print(f"Title: {opportunity.get('title')}")
            print(f"Description: {opportunity.get('description')}")
            print(f"URL: {opportunity.get('url')}")
            print(f"Organization: {opportunity.get('organization', {}).get('name')}")
            print("Activities:", ", ".join([a.get("name") for a in activities]))
            print("📍 Regions:", opportunity.get("audience", {}).get("regions", []))
            print("-" * 60)

else:
    print(f"❌ Error: Received status code {response.status_code}")